# Kubectl

Check you can connect to the Kubernetse cluster by getting the client and server versions, and listing the nodes in the cluster. If this doesn't work ask for help!

In [1]:
kubectl version
kubectl get nodes

Client Version: version.Info{Major:"1", Minor:"10", GitVersion:"v1.10.4", GitCommit:"5ca598b4ba5abb89bb773071ce452e33fb66339d", GitTreeState:"clean", BuildDate:"2018-06-06T08:13:03Z", GoVersion:"go1.9.3", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"9", GitVersion:"v1.9.2+coreos.0", GitCommit:"b427929b2982726eeb64e985bc1ebb41aaa5e095", GitTreeState:"clean", BuildDate:"2018-01-18T22:56:14Z", GoVersion:"go1.9.2", Compiler:"gc", Platform:"linux/amd64"}
NAME            STATUS    ROLES     AGE       VERSION
ome-lochy-m01   Ready     master    169d      v1.9.2+coreos.0
ome-lochy-n01   Ready     node      169d      v1.9.2+coreos.0
ome-lochy-n02   Ready     node      169d      v1.9.2+coreos.0


List namespaces

In [2]:
kubectl get ns

NAME            STATUS    AGE
default         Active    169d
idr-redmine     Active    7d
jupyter-int     Active    152d
jupyter-train   Active    88d
kube-public     Active    169d
kube-system     Active    169d
monitoring      Active    43d
nginx-ingress   Active    41d


List pods in the jupyter-int namespace

In [3]:
kubectl -n jupyter-int get pods

NAME                     READY     STATUS    RESTARTS   AGE
hub-6f4fcc7b-gl2wr       1/1       Running   0          7m
jupyter-spli             1/1       Running   0          1h
proxy-5d7977cdc7-ftjs4   1/1       Running   0          2h


`hub-*` an `proxy-*` are the two JupyterHub server pods. Get the logs for the past 2 minutes from the hub

In [4]:
HUBPOD=$(kubectl -n jupyter-int get pods | grep hub | cut -d\  -f1)
echo $HUBPOD
kubectl -n jupyter-int logs $HUBPOD --since 2m

hub-6f4fcc7b-gl2wr
[D 2018-07-31 17:36:20.451 JupyterHub proxy:670] Proxy: Fetching GET http://10.233.26.9:8001/api/routes
[D 2018-07-31 17:36:20.453 JupyterHub proxy:724] Omitting non-jupyterhub route '/'
[I 2018-07-31 17:36:20.453 JupyterHub proxy:301] Checking routes
[I 2018-07-31 17:36:29.393 JupyterHub log:158] 302 GET /hub/metrics -> /jupyterhub-internal/hub/hub/metrics (@10.233.64.78) 0.59ms
[W 2018-07-31 17:36:29.394 JupyterHub log:158] 404 GET /jupyterhub-internal/hub/hub/metrics (@10.233.64.78) 1.00ms
[D 2018-07-31 17:37:20.451 JupyterHub proxy:670] Proxy: Fetching GET http://10.233.26.9:8001/api/routes
[D 2018-07-31 17:37:20.453 JupyterHub proxy:724] Omitting non-jupyterhub route '/'
[I 2018-07-31 17:37:20.453 JupyterHub proxy:301] Checking routes
[I 2018-07-31 17:37:29.393 JupyterHub log:158] 302 GET /hub/metrics -> /jupyterhub-internal/hub/hub/metrics (@10.233.64.78) 0.57ms
[W 2018-07-31 17:37:29.394 JupyterHub log:158] 404 GET /jupyterhub-internal/hub/hub/metrics (@10.233

The JupyterHub pod is managed by a deployment object, so the pod name may change. Since the deployment only runs a single instance of the main JupyterHub pod you can refer to it this way

In [5]:
kubectl -n jupyter-int logs deploy/hub --since 2m

[D 2018-07-31 17:36:20.451 JupyterHub proxy:670] Proxy: Fetching GET http://10.233.26.9:8001/api/routes
[D 2018-07-31 17:36:20.453 JupyterHub proxy:724] Omitting non-jupyterhub route '/'
[I 2018-07-31 17:36:20.453 JupyterHub proxy:301] Checking routes
[I 2018-07-31 17:36:29.393 JupyterHub log:158] 302 GET /hub/metrics -> /jupyterhub-internal/hub/hub/metrics (@10.233.64.78) 0.59ms
[W 2018-07-31 17:36:29.394 JupyterHub log:158] 404 GET /jupyterhub-internal/hub/hub/metrics (@10.233.64.78) 1.00ms
[D 2018-07-31 17:37:20.451 JupyterHub proxy:670] Proxy: Fetching GET http://10.233.26.9:8001/api/routes
[D 2018-07-31 17:37:20.453 JupyterHub proxy:724] Omitting non-jupyterhub route '/'
[I 2018-07-31 17:37:20.453 JupyterHub proxy:301] Checking routes
[I 2018-07-31 17:37:29.393 JupyterHub log:158] 302 GET /hub/metrics -> /jupyterhub-internal/hub/hub/metrics (@10.233.64.78) 0.57ms
[W 2018-07-31 17:37:29.394 JupyterHub log:158] 404 GET /jupyterhub-internal/hub/hub/metrics (@10.233.64.78) 0.95ms


You can also use selectors to select one or more pods. For example, get some information about a deployment which may include labels attached to the pods

In [6]:
kubectl -n jupyter-int get deploy
kubectl -n jupyter-int describe deploy hub

NAME      DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
hub       1         1         1            1           32d
proxy     1         1         1            1           32d
Name:               hub
Namespace:          jupyter-int
CreationTimestamp:  Fri, 29 Jun 2018 13:52:08 +0000
Labels:             app=jupyterhub
                    chart=jupyterhub-v0.7-578b3a2
                    component=hub
                    heritage=Tiller
                    release=jupyter-int
Annotations:        deployment.kubernetes.io/revision=8
Selector:           app=jupyterhub,component=hub,release=jupyter-int
Replicas:           1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:       Recreate
MinReadySeconds:    0
Pod Template:
  Labels:           app=jupyterhub
                    component=hub
                    hub.jupyter.org/network-access-proxy-api=true
                    hub.jupyter.org/network-access-proxy-http=true
                    hub.jupyter.org/network-ac

Select all pods with the label `component=hub`

In [7]:
kubectl -n jupyter-int get pods --selector=component=hub

NAME                 READY     STATUS    RESTARTS   AGE
hub-6f4fcc7b-gl2wr   1/1       Running   0          7m


If for some reason you need to restart the hub the easiest way is to delete it
and it will be recreated automatically
```
kubectl -n jupyter-int delete pod --selector=component=hub
kubectl -n jupyter-int get pods
```